In [1]:
# Case 2
# Метрики на основе данных
# Monetization:
#     Revenue (выручка)
#     ARPU (Средний чек  на пользователя)
#     ARPPU (Средний чек на платящего пользователя)
# Behavior:
#     Average cnt of session (Среднее кол-во сессий на пользвоателя)
#     In-game time spent (Среднее время проведенное в игре по пользователю)
# Conversion

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
import from_db_library as export
df = export.export_data_from_db('table','schema','user','pwd','host','port')

In [ ]:
# Добавим период: до/во время/после акции
df['Period'] = pd.cut(
    df['Date'],
    bins=[pd.Timestamp('2025-05-01'), pd.Timestamp('2025-05-10'),
          pd.Timestamp('2025-05-13'), pd.Timestamp('2025-05-21')],
    labels=['Before', 'During', 'After']
)

# Базовые агрегаты
daily_metrics = df.groupby(['Date', 'Period']).agg({
    'User_ID': 'nunique',
    'Revenue': 'sum',
    'Payments': 'sum',
    'Sessions': 'sum',
    'Playing_Time': 'sum'
}).reset_index().rename(columns={'User_ID': 'Unique_Users'})

# Добавим базовые метрики
daily_metrics['ARPU'] = daily_metrics['Revenue'] / daily_metrics['Unique_Users']
daily_metrics['Avg_Sessions'] = daily_metrics['Sessions'] / daily_metrics['Unique_Users']
daily_metrics['Avg_Playing_Time'] = daily_metrics['Playing_Time'] / daily_metrics['Unique_Users']

In [ ]:
# Агрегация по периодам
period_summary = df.groupby('Period').agg({
    'User_ID': pd.Series.nunique,
    'Revenue': 'sum',
    'Payments': 'sum'
}).rename(columns={'User_ID': 'Unique_Users'})

period_summary['ARPU'] = period_summary['Revenue'] / period_summary['Unique_Users']
period_summary['ARPPU'] = period_summary['Revenue'] / df[df['Revenue'] > 0].groupby('Period')['User_ID'].nunique()
period_summary['Conversion_Rate'] = df[df['Revenue'] > 0].groupby('Period')['User_ID'].nunique() / period_summary['Unique_Users']


In [ ]:
# Сегментация
# Возраст аккаунта на момент даты
df['User_Age_Days'] = (df['Date'] - df['Reg_Date']).dt.days

# Когорты по возрасту
df['Cohort'] = pd.cut(df['User_Age_Days'], bins=[-1, 7, 30, 90, 10000], labels=['0-7d', '8-30d', '31-90d', '90+d'])

# Группировка
cohort_platform = df.groupby(['Period', 'Platform', 'Cohort']).agg({
    'User_ID': 'nunique',
    'Revenue': 'sum'
}).reset_index()

In [4]:
# Визуализация

In [ ]:
# Выручка по дням
plt.figure(figsize=(12,6))
sns.lineplot(data=daily_metrics, x='Date', y='Revenue', hue='Period')
plt.title('Динамика выручки по дням')
plt.xlabel('Дата')
plt.ylabel('Выручка')
plt.grid(True)
plt.tight_layout()

In [ ]:
# ARPU\ARPPU по периодам
sns.barplot(data=period_summary.reset_index(), x='Period', y='ARPU')
plt.title('ARPU по периодам')

In [ ]:
# Тепловая карта конверсии по платформам и когортам
pivot = cohort_platform.pivot_table(index='Cohort', columns='Platform', values='Revenue', aggfunc='sum')
sns.heatmap(pivot, annot=True, fmt=".0f", cmap='YlGnBu')
plt.title('Суммарная выручка по когортам и платформам')